# Session 2 - Demo 2.4 - Data-Augmented Question Answering

<a href="https://colab.research.google.com/github/dair-ai/pe-for-llms/blob/main/notebooks/session-2/demo-2.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade langchain
!pip install --upgrade python-dotenv
!pip install chromadb

In [1]:
import openai
import os
import IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()

# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

# for LangChain
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

First, we need to download the data we want to use as source to augment generation.

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

As our data source, we will use a transcription of Karpathy's recent lecture on GPT. 

In [29]:
#with open('../data/state_of_the_union.txt') as f:
with open('../data/kar-gpt.txt') as f:
    text_data = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator=" ")
texts = text_splitter.split_text(text_data)

embeddings = OpenAIEmbeddings()

In [31]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])

Using embedded DuckDB without persistence: data will be transient


In [34]:
query = "What is the course about?"
docs = docsearch.similarity_search(query)

In [10]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

In [37]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
query = "What is the course about?"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' This course is about understanding and appreciating how chat GPT works, and how to develop a transformer neural network. It requires proficiency in Python and some basic understanding of calculus and statistics.\nSOURCES: 1, 7, 107, 108'}

In [38]:
template = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.
Respond in Spanish.

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER IN SPANISH:"""

# create a prompt template
PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])

# query 
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
query = "What is the course about?"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\nEl curso trata sobre cómo entrenar un modelo de lenguaje usando una red neuronal de transformadores. Se requiere una proficiencia en Python y una comprensión básica de cálculo y estadística. El curso también cubre el ajuste fino, que puede ser un ajuste supervisado simple o algo más sofisticado como el entrenamiento de un modelo de recompensa y luego varias rondas de PPO para alinearlo con el modelo de recompensa.\n\nFUENTES:\n108, 1, 107, 7'}